## **Prueba con modelo de regresión logística**


In [1]:
#Leer los archivos
import pandas as pd
import numpy as np
# Procesar texto y entrenar el modelo
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression

In [2]:
# Guardando el dataset como un dataframe
df = pd.read_csv("/content/df16to25.csv")
df.head()

,id,stars,review_body,review_title,language,product_category,lenght_review_body,lenght_review_title,lenght_product_category
0,151766,4,Mando aceptable precio un poco caro.. pero cum...,Cumple su funcion,es,electronics,134,17,11
1,10209,5,Muy chula y se ha pegado muy bien al maletero ...,Bruja grande,es,automotive,75,12,10
2,197887,5,Me gustó mucho como idea para regalos funcionó...,Idea genial,es,home,51,11,4
3,68193,1,"A los seis meses de uso deja de funcionar, es ...",Decepcionante,es,wireless,64,13,8
4,149127,3,"Para un rato vale,pero no agarra mucho y para ...",Que para algunos juegos no vale,es,video_games,97,31,11


**Creación de variable Sentimiento**

In [3]:
# Crendo la variable Sentimiento a partir de las tres columnas importantes (stars,review_body,lenght_review_body)
df = df[['stars', 'review_body', 'lenght_review_body']]
df.dropna(inplace=True)

def sentimiento(stars):
    if stars <= 2:
        return 'Negativo'
    elif stars == 3:
        return 'Neutro'
    else:
        return 'Positivo'

df['sentiment'] = df['stars'].apply(sentimiento)

**Tratamiento de validación y normalización de los datos**

In [4]:
#Se importa la biblioteca re para eliminar símbolos, validación de datos, normalizar texto y reducir el ruido, con esto se pretende evitar que el modelo muestre Neutro tan seguido
import re

def limpiar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'[^a-záéíóúñ ]', '', texto)
    texto = texto.strip()
    return texto

df['review_clean'] = df['review_body'].apply(limpiar_texto)

In [5]:
# Esta vectorización intentará hacer enfasís en las palabras identificar mejor las positivas y negativas
vectorizer = TfidfVectorizer(
    max_features=500,
    ngram_range=(1, 2),   # frases cortas
    min_df=1
)

X_text = vectorizer.fit_transform(df['review_clean'])
X_len = df[['lenght_review_body']].values

from scipy.sparse import hstack
X = hstack([X_text, X_len])

y = df['sentiment']

In [6]:
#Split correcto (estratificado) dividir matrices en sub conjuntos aleatorios de prueba y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

**Modelo de regresión logística**

In [7]:
# Se crea el modelo de regresión logística | Este debería funcionar mejor porque aprende pesos para palabras negativas, generaliza mejor con pocos datos
model = LogisticRegression(
    max_iter=1000,
    class_weight='balanced'
)

model.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [8]:
# La logíca para decidir el sentimiento esta un poco mejor trabajada. Ahora el modelo intentará tomar más responsabilidad y no optar tanto por dar respuesta Neutra
def decidir_sentimiento(probs, margen=0.10):
    neg = probs.get('Negativo', 0)
    neu = probs.get('Neutro', 0)
    pos = probs.get('Positivo', 0)

    # Regla 1: Negativo claramente dominante
    if neg > pos + margen and neg > neu + margen:
        return 'Negativo'

    # Regla 2: Positivo claramente dominante
    if pos > neg + margen and pos > neu + margen:
        return 'Positivo'

    # Si no hay dominancia clara → Neutro
    return 'Neutro'

In [9]:
# Se realizo el entrenamiento del modelo con la libreria metrics se guarda el accuracy del modelo
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

In [10]:
# Creación de un pequeño diccionario de palabras clave. El proposito es ayudar a que el modelo logré identificar estas palabras de acuerdo a su sentimiento correspondiente. Tambien, si alguna palabra desconocida se encuentra acompañada de alguna de estas, pueda realizar la relación.
positivas_fuertes = {
    "bien","genial","buen","agrado","excelente", "increible", "increíble", "gusto","magnifico",
    "perfecto", "lindo", "bueno", "maravilloso", "bonito","espectacular","agradable"
}

negativas_fuertes = {
    "mal","malo", "mala","malisimo","terrible", "horrible","desastroza","lamentable"
    "pesimo", "pésimo", "defectuoso","espantoso","feo","desfavorables","desagradable"
}

**Regla de probabilidad para ayudar al modelo**

In [11]:
# Es una regla de decisión que se usa después de que el modelo ya calculó probabilidades. Ayuda a evitár decisiones forzadas cuando el modelo está inseguro.
def decidir_sentimiento(probs, margen=0.20):
    neg = probs['Negativo']
    neu = probs['Neutro']
    pos = probs['Positivo']

    # Positivo claramente dominante
    if pos > neg + margen and pos > neu + margen:
        return 'Positivo'

    # Negativo claramente dominante
    if neg > pos + margen and neg > neu + margen:
        return 'Negativo'

    # Si no hay dominancia clara
    return 'Neutro'

**Código Final del Modelo de regresión logística**

In [12]:
def predecir_sentimiento(texto):
    texto_limpio = limpiar_texto(texto)
    palabras = set(texto_limpio.split())

    # 1️⃣ Regla léxica (prioridad)
    if palabras & positivas_fuertes:
        return f"Sentimiento: Positivo | Precisión del modelo: {round(accuracy, 2)}"

    if palabras & negativas_fuertes:
        return f"Sentimiento: Negativo | Precisión del modelo: {round(accuracy, 2)}"

    # 2️⃣ Modelo ML
    texto_vec = vectorizer.transform([texto_limpio])
    longitud = np.array([[len(texto_limpio)]])
    X_nuevo = hstack([texto_vec, longitud])

    probs_raw = model.predict_proba(X_nuevo)[0]
    clases = model.classes_
    probs = {clase: float(prob) for clase, prob in zip(clases, probs_raw)}

    # 3️⃣ Decisión por dominancia
    sentimiento = decidir_sentimiento(probs)

    return f"Sentimiento: {sentimiento} | Precisión del modelo: {round(accuracy, 2)}"

**Prueba del Modelo**

In [26]:
#Se realiza la prueba del modelo. La precisión se mantiene alrededor del 0.50 al 0.55, lo cual es positivo para este modelo básico.
predecir_sentimiento("el producto estaba sucio")

'Sentimiento: Negativo | Precisión del modelo: 0.5'

**Métricas** Precisión(Accuracy), Recall(Sensibilidad), F1-Score(Puntuación única(de 0 a 1) que indica el rendimiento general de un modelo de clasificación).

In [14]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Negativo       0.33      0.33      0.33         6
      Neutro       0.50      0.60      0.55         5
    Positivo       0.62      0.56      0.59         9

    accuracy                           0.50        20
   macro avg       0.49      0.50      0.49        20
weighted avg       0.51      0.50      0.50        20



**Conclusión del modelo**

El sistema del modelo utiliza un enfoque híbrido. Primero aplica reglas léxicas para identificar palabras con polaridad evidente. Si no las detecta, el comentario se transforma en características numéricas y se evalúa con un modelo supervisado. La decisión final se basa en probabilidades, asignando un sentimiento solo cuando existe dominancia clara; de lo contrario, se clasifica como neutro.

In [15]:
#Biblioteca para guardar (serializar) el modelo de Regresión Logística
import joblib

In [16]:
print(locals().keys())

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'pd', 'np', 'train_test_split', 'TfidfVectorizer', 'classification_report', 'accuracy_score', 'LogisticRegression', '_i2', 'df', '_2', '_i3', 'sentimiento', '_i4', 're', 'limpiar_texto', '_i5', 'vectorizer', 'X_text', 'X_len', 'hstack', 'X', 'y', '_i6', 'X_train', 'X_test', 'y_train', 'y_test', '_i7', 'model', '_7', '_i8', 'decidir_sentimiento', '_i9', 'y_pred', 'accuracy', '_i10', 'positivas_fuertes', 'negativas_fuertes', '_i11', '_i12', 'predecir_sentimiento', '_i13', '_13', '_i14', '_i15', 'joblib', '_i16'])


In [17]:
# Guardando el modelo de regresión
joblib.dump(model, 'modelo_regresion_logistica.pkl')

['modelo_regresion_logistica.pkl']

In [18]:
# Guardando el vectorizer
joblib.dump(vectorizer, 'vectorizer_regresion_logistica.pkl')

['vectorizer_regresion_logistica.pkl']

In [19]:
#Guardar el accuracy (Precisión)
joblib.dump(accuracy, 'accuracy_regresion_logistica.pkl')

['accuracy_regresion_logistica.pkl']

In [20]:
# Comprobando que el modelo y lo demás alla quedado guardado
print(type(model))
print(type(vectorizer))
print(accuracy)

<class 'sklearn.linear_model._logistic.LogisticRegression'>
<class 'sklearn.feature_extraction.text.TfidfVectorizer'>
0.5


In [21]:
import os
os.listdir()

['.config',
 'accuracy_regresion_logistica.pkl',
 'df16to25.csv',
 'modelo_regresion_logistica.pkl',
 'vectorizer_regresion_logistica.pkl',
 'sample_data']

In [22]:
# Cargar el Modelo
model = joblib.load('modelo_regresion_logistica.pkl')
vectorizer = joblib.load('vectorizer_regresion_logistica.pkl')
accuracy = joblib.load('accuracy_regresion_logistica.pkl')

In [28]:
predecir_sentimiento("me gusto mucho este producto")

'Sentimiento: Positivo | Precisión del modelo: 0.5'